In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle

In [2]:
os.chdir("C:\\Users\\Admin\\FingerprintClassification\\model")
from preprocess.preprocessing3 import preprocessing_normal

In [75]:
def read_data(label2num):
    id_anh = 0
    X=[]
    Y=[]
    #output_folder = "C:\\Users\\ADMIN\\FingerprintClassification\\model\\preprocessed_fingerprint"
    for label in os.listdir(os.path.join("preprocessed_fingerprint")):
        id_anh+=1
        img = cv2.imread(os.path.join('preprocessed_fingerprint',str(id_anh)+'.png'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint"])
        if id_anh>=4000: break
    for label in os.listdir(os.path.join("preprocessed_fingerprint")):
        id_anh+=1
        img = cv2.imread(os.path.join('preprocessed_fingerprint',str(id_anh)+'.png'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint_noise"])
        if id_anh>=8000: break
    return X,Y


In [76]:
def read_data(label2num):
    X = []
    Y = []
    for label in os.listdir(os.path.join("fingerprint")):
        img = cv2.imread(os.path.join('fingerprint',label))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
        img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint"])
    for label in os.listdir(os.path.join("fingerprint_noise")):
        img = cv2.imread(os.path.join('fingerprint_noise',label))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
        img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint_noise"])
    return X,Y

In [77]:
label2num = {'fingerprint_noise':0, 'fingerprint':1}
X, Y = read_data(label2num)
print(len(X))
print(len(Y))

8000
8000


In [24]:
print(X[0].shape)

(256, 256)


In [ ]:
plt.imshow(X[7750], cmap='gray')
print(X[7750].shape)
plt.axis("off")

In [26]:
from skimage import feature

In [54]:
cell_size = (8, 8)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins
def hist_vec(X):
    vector_HoG = []
    for i in range(len(X)):
        img = X[i]
        img = cv2.GaussianBlur(img, (3,3), 0)
        # 1. Khai báo các tham số
        #cell_size = (8, 8)  # h x w in pixels
        #block_size = (2, 2)  # h x w in cells
        #nbins = 9  # number of orientation bins

        # 2. Tính toán các tham số truyền vào HOGDescriptor
        # winSize: Kích thước của bức ảnh được crop để chia hết cho cell size.
        # blockSize: Kích thước của 1 block
        blockSize = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
        # blockStride: Số bước di chuyển của block khi thực hiện chuẩn hóa histogram bước 3
        blockStride = (cell_size[1], cell_size[0])

        # 3. Compute HOG descriptor
        hog = cv2.HOGDescriptor(_winSize=img.shape,
                                _blockSize=blockSize,
                                _blockStride=blockStride,
                                _cellSize=cell_size,
                                _nbins=nbins)

        # Kích thước của lưới ô vuông.
        n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])
        
        # Reshape hog feature
        hog_feats = hog.compute(img)\
                    .reshape(n_cells[1] - block_size[1] + 1,
                                n_cells[0] - block_size[0] + 1,
                                block_size[0], block_size[1], nbins) \
                    .transpose((1, 0, 2, 3, 4))  
        #H = feature.hog(img, orientations=9, pixels_per_cell=(8, 8),
        #            cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
        H = hog_feats.flatten()
        vector_HoG.append(H)
        #print('Kích thước hog features: ', H.shape)
    return vector_HoG

In [55]:
vector_HoG = hist_vec(X)

In [56]:
print('Kích thước hog features: ', vector_HoG[0].shape)

Kích thước hog features:  (34596,)


In [61]:
from sklearn.decomposition import PCA
def pca(image_descriptors):
    
    pca = PCA(n_components=100)

    # Fit và transform dữ liệu
    pca.fit(image_descriptors)
    des_pca = pca.transform(image_descriptors)
    return des_pca

In [62]:
pca_HoG = pca(vector_HoG)

In [64]:
print('Kích thước hog features: ', pca_HoG[0].shape)

Kích thước hog features:  (100,)


In [36]:
from sklearn.model_selection import train_test_split, learning_curve, KFold

In [79]:
X_train = []
X_test = []
Y_train = []
Y_test = []
X_valid = []
Y_valid = []
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)

print(len(X_train), " ",len(X_valid), " ", len(X_test))

4800   1600   1600


In [50]:
from sklearn.metrics import accuracy_score

In [80]:
# Khởi tạo KFold với k=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Khởi tạo mô hình (ví dụ: Random Forest)
model = RandomForestClassifier()

# Lưu trữ kết quả
results = []

for train_index, val_index in kf.split(X_train):
    #print(val_index)
    # Chia tập train thành train và validation cho mỗi fold
    #X_train_fold, X_val_fold = X_train[np.array(train_index)], X_train[np.array(val_index)]
    #y_train_fold, y_val_fold = Y_train[np.array(train_index)], Y_train[np.array(val_index)]
    X_train_fold = []
    y_train_fold = []
    X_val_fold = []
    y_val_fold = []
    for sg in train_index:
        X_train_fold.append(X_train[sg])
        y_train_fold.append(Y_train[sg])
    #print(len(X_train_fold), len(X_val_fold), len(y_train_fold), len(y_val_fold))
    for sg in val_index:
        #print(sg)
        X_val_fold.append(X_train[sg])
        y_val_fold.append(Y_train[sg])
    # Huấn luyện mô hình trên tập train_fold
    model.fit(X_train_fold, y_train_fold)

    # Đánh giá trên tập validation
    y_val_pred = model.predict(X_val_fold)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)
    print(f"Validation accuracy: {val_accuracy}")

    # Đánh giá trên tập test (độc lập)
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(Y_valid, y_test_pred)
    print(f"True Vali accuracy: {test_accuracy}")

    results.append({'val_accuracy': val_accuracy, 'test_accuracy': test_accuracy})

# In kết quả trung bình
print("Average validation accuracy:", sum(result['val_accuracy'] for result in results) / len(results))
print("Average true vali accuracy:", sum(result['test_accuracy'] for result in results) / len(results))

Validation accuracy: 0.8125
True Vali accuracy: 0.5
Validation accuracy: 0.8197916666666667
True Vali accuracy: 0.5025
Validation accuracy: 0.8354166666666667
True Vali accuracy: 0.51
Validation accuracy: 0.8166666666666667
True Vali accuracy: 0.500625
Validation accuracy: 0.78125
True Vali accuracy: 0.4975
Average validation accuracy: 0.813125
Average true vali accuracy: 0.502125


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.svm import SVC

In [72]:
lgbm = lgb.LGBMClassifier(class_weight="balanced").fit(X_train, Y_train)
print(lgbm.score(X_train, Y_train))
print(lgbm.score(X_test, Y_test))

[LightGBM] [Info] Number of positive: 3196, number of negative: 3204
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.197055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 918000
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 3600
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
0.99953125
0.759375


In [33]:
log = LogisticRegression().fit(X_train, Y_train)
print(log.score(X_train, Y_train))
print(log.score(X_test, Y_test))

0.8641666666666666
0.783125


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
clf = AdaBoostClassifier(algorithm='SAMME')
clf.fit(X_train, Y_train)
print(clf.score(X_train, Y_train))
print(clf.score(X_test, Y_test))

0.7629166666666667
0.7525


In [41]:
dtree = DecisionTreeClassifier().fit(X_train, Y_train)
print(dtree.score(X_train, Y_train))
print(dtree.score(X_test, Y_test))

1.0
0.649375


In [67]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,Y_train)
print(knn.score(X_train, Y_train))
print(knn.score(X_test, Y_test))

0.7691666666666667
0.57625


In [68]:
rf = RandomForestClassifier(n_estimators=12,max_depth=20)
rf.fit(X_train, Y_train)
print(rf.score(X_train, Y_train))
print(rf.score(X_test, Y_test))

0.9927083333333333
0.645625


In [45]:
sv = SVC(kernel = 'linear')
sv.fit(X_train, Y_train)
print(sv.score(X_train, Y_train))
print(sv.score(X_test, Y_test))

1.0
0.74


In [ ]:
sv = SVC(kernel = 'rbf')
sv.fit(X_train, Y_train)
print(sv.score(X_train, Y_train))
print(sv.score(X_test, Y_test))